# Импорт

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import numpy as np
import random
import datetime
from datetime import datetime
import warnings
import os, fnmatch
import yadisk
import shutil
import psycopg2
from sqlalchemy import create_engine
from matplotlib import pyplot as plt 
import seaborn as sns
from tqdm import tqdm

warnings.simplefilter('ignore')
options = webdriver.ChromeOptions()
now = datetime.now()
options.headless = True

token = '*****'
dir_ = "*****"
EXE_PATH = r'C:\chromedriver.exe'

dates = {
    'января':'01',
    'февраля':'02',
    'марта':'03',
    'апреля':'04',
    'мая':'05',
    'июня':'06',
    'июля':'07',
    'августа':'08',
    'сентября':'09',
    'октября':'10',
    'ноября':'11',
    'декабря':'12'
}

host_2 = '*****'
user_2 = '*****'
password_2 = '*****'
db_name_2 = '*****'
port_2 = 5432

connection_2 = psycopg2.connect(
host=host_2,
    user=user_2,
    password=password_2,
    database = db_name_2,
    port=port_2
)


connection_2.autocommit = True

SQL_insert = "INSERT INTO cars_ishevsk (mark, model, kuzov, year, km, engine_volume, engine_type, KPP, privod, complectation, color, owners, price, date, ad_number, url_link, description)\
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s,%s);"

db_config = {'user': '*****', # имя пользователя
'pwd': '*****', # пароль
'host': '*****',
'port': 5432, # порт подключения
'db': '*****'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

engine = create_engine(connection_string)
y = yadisk.YaDisk(token=token)
print(y.check_token())



True


# Функции

In [2]:
def get_date_type(date):
    try:
        return datetime.strptime(date, "%d.%m.%Y")
    except:        
        return datetime.strptime(date[:10], "%Y-%m-%d")

def get_pages_number():
    pages = driver.find_elements("class name", "pagination-item-JJq_j")
    pages_ = []
    for page in pages:    
        if page.text.isdigit():        
            pages_.append(int(page.text))
    return(max(pages_))

def get_norm_links():
    all_links = []
    elems = driver.find_elements("xpath" ,"//a[@href]")
    for elem in elems:
        all_links.append(elem.get_attribute("href"))
    norm_links = []
    for link in all_links:
        if link.split('_')[-1].isdigit():
            norm_links.append(link)
    return(list(set(norm_links)))

def get_price():
    try:
        try:
            x = driver.find_element("class name", "style-item-view-price-content-1aHxn")
            return float(x.text.replace('\n', '').replace('₽', '').replace(' ', ''))
        except:
            pass
        try:
            x = driver.find_element("class name", "style-item-price-PuQ0I")
            price = ''
            ch_ = x.text.replace('\n', '').replace('₽', '').replace(' ', '')
            for ch in ch_:
                if ch.isdigit():
                    price += ch
            return(float(price))
        except:
            pass
    
    except:        
        return(None)

def yandex():
    print('Начинаем перенос скринов на Я.диск')
    # Собираем файлы
    files = []
    listOfFiles = os.listdir('screenshots')  
    pattern = "*.png"  
    for entry in listOfFiles:  
        if fnmatch.fnmatch(entry, pattern):            
                files.append(entry)
    # переносим в другую директорию
    for file in files:
        shutil.move('screenshots' +'\\' + file, os.getcwd())
    #записываем на Я.Диск
    c = 0        
    for file in files:
        print(f'Записываем {file}')
        if y.is_file(f'{dir_}{file}'):
            print('Такой уже есть')
            continue
        else:
            c+=1
            y.upload(file, f'{dir_}{file}')
    #удаляем файлы       
    files = []
    listOfFiles = os.listdir('.')  
    pattern = "*.png"  
    for entry in listOfFiles:  
        if fnmatch.fnmatch(entry, pattern):            
                files.append(entry)
    for file in files:
        os.remove(file)

    print(f'{c} файлов записано на Я.Диск')

def is_bad_adv(i,j):
    try:
        driver.find_element("class name", "desktop-16c9mq7").text
        print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
        print(url)
        print('Страницы не существует')
        return(True)
    except:        
        try:
            driver.find_element("class name", "closed-warning-content-2ooy4").text
            print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
            print('Снято с публикации')
            print(url)
            return(True)
        except:
            try:
                driver.find_element("class name", "page-title-count-wQ7pG").text
                print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
                print(url)
                print('Страница поиска, объявления нет')
                return(True)
            except:
                    try:
                        driver.find_element("class name", "b-404").text
                        print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
                        print('Нет такой страницы')
                        print(url)
                        return(True)
                    except:
                        try:
                            driver.find_element("class name", "closed-warning-content-_f4_B").text
                            print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
                            print(url)
                            print('Снято с публикации')
                            return(True)
                        except:
                            return False
                    
def get_params():
    try:
        if len(driver.find_elements("class name", "params-paramsList__item-1Xeok")) > 0:
            params = driver.find_elements("class name", "params-paramsList__item-1Xeok")
            params_ = []
            for param in params:
                params_.append(param.text.split(':'))
            param_list = []
            for param in params_:
                  param_list.extend(param)
            return param_list
        
        if len(driver.find_elements("class name", "params-paramsList-zLpAu")) > 0:
            param_list = driver.find_elements("class name", "params-paramsList-zLpAu")[0].text.replace('\n', ' ').split(' ')
            return param_list
    except:
        print('Ошибка в параметрах')
        print(url)
        return('ERRROR')
    
def get_model():
    try:
        test = driver.find_element("class name", "breadcrumbs-root-_GADZ")
        return test.text.split('·')[-2]
    except:
        return(None)
    
def get_mark():
    try:
        test = driver.find_element("class name", "breadcrumbs-root-_GADZ")
        return test.text.split('·')[-3]
    except:
        return(None)
    
def get_year(params):
    try:
        return params[params.index('выпуска:')+1]
    except:
        return None

def get_probeg(params):
    try:
        return params[params.index('Пробег:')+1]
    except:
        return None

def get_dvigatel(params):
    try:
        return params[params.index('двигателя:')+1]
    except:
        return None
    
def get_dvig_type(params):
    try:
        return params[params.index('Тип')+2]
    except:
        return None
    
def get_kuzov_type(params):
    try:
        return params[params.index('кузова:')+1]
    except:
        return None

def get_complectation(params):
    try:
        return params[params.index('Комплектация:')+1]
    except:
        return None
    
def get_kpp(params):
    try:
        return params[params.index('передач:')+1]
    except:
        return None
    
def get_color(params):
    try:
        return params[params.index('Цвет:')+1]
    except:
        return None
    
def get_vladelcev(params):
    try:
        return params[params.index('Владельцев')+3]
    except:
        return None
    
def get_privod(params):
    try:
        return params[params.index('Привод:')+1]
    except:
        return None

def get_date():
    try:
        #1 ексепт
        try:
            test = driver.find_element("class name", "style-item-metadata-date-1y5w6")
            return(' '.join(test.text.split()[:2]) + ' ' + str(time.localtime().tm_year))
        except:
            pass
        
        # 2 ексепт
        try:
            test = driver.find_element("class name", "style-item-metadata-date-EEthc")
            return(' '.join(test.text.split()[:2]) + ' ' + str(time.localtime().tm_year))
        except:
            pass
        
        #3 ексепт
        try:
            test = driver.find_element("class name", "style-item-footer-3uXQz")
            if 'сегодн' in test.text.split()[3]:
                return(str(now.day) +'.'+  str(now.month)+'.'+ str(now.year))
            if 'вчер' in test.text.split()[3]:                
                return(str(now.day-1) +'.'+  str(now.month)+'.'+ str(now.year))
            else:
                date = (test.text.split()[3]+ ' '+ test.text.split()[4]+' '+ str(now.year))
                date = date.split()
                date[1] = dates[date[1]]
                return('.'.join(date))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-footer-Ufxh_")
            if 'сегодн' in test.text.split('·')[1].strip():
                return(str(now.day) +'.'+  str(now.month)+'.'+ str(now.year))
            if 'вчер' in test.text.split('·')[1].strip():                
                return(str(now.day-1) +'.'+  str(now.month)+'.'+ str(now.year))
            else:
                date = (test.text.split()[3]+ ' '+ test.text.split()[4]+' '+ str(now.year))
                date = date.split()
                date[1] = dates[date[1]]
                return('.'.join(date))
        except:
            pass
    
    except:        
        return(None)
    
def get_view():
    try:
        try:
            test = driver.find_element("class name", "style-item-description-html-1_RNo")
            return(test.text.replace('\n', ' '))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-description-text-SzN56")
            return(test.text.replace('\n', ' '))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-description-text-mc3G6")
            return(test.text.replace('\n', ' '))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-description-html-qCwUL")
            return(test.text.replace('\n', ' '))
        except:
            pass
    except:        
        return(None)
    


# Links

In [ ]:
driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
driver.get('https://www.avito.ru/izhevsk/avtomobili/s_probegom-ASgBAgICAUSGFMjmAQ?f=ASgBAQICAkTyCrCKAYYUyOYBAkCm2g4UAvrwDxTou_cC&p=1&radius=0')
page_numbers = get_pages_number()
driver.close()
print(f'Количество страниц с ссылками: {page_numbers}')

links = []


for i in tqdm(range(1, page_numbers+1)):
    driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
    driver.get(f'https://www.avito.ru/izhevsk/avtomobili/s_probegom-ASgBAgICAUSGFMjmAQ?f=ASgBAQICAkTyCrCKAYYUyOYBAkCm2g4UAvrwDxTou_cC&p={i}&radius=0')
    links.extend(get_norm_links())
    driver.close()
    time.sleep(random.randint(5,12))
    
with connection_2.cursor() as cursor:
                    cursor.execute(
                    'TRUNCATE TABLE cars_links'
                    )
        
query = "SELECT * FROM cars_ishevsk"
old_links = list(pd.io.sql.read_sql(query, con = engine)['url_link'].values)

for link in links:
    if 'izhevsk' in link and link not in old_links:
        with connection_2.cursor() as cursor:
                    cursor.execute(
                    f'''INSERT INTO cars_links (links) VALUES ('{link}')'''
                    )
    else:
        continue
        
query = "SELECT links FROM cars_links"
print(f'Новых линков: {len(list(pd.io.sql.read_sql(query, con = engine)["links"].values))}')

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Количество страниц с ссылками: 100


 22%|█████████████████▊                                                               | 22/100 [17:49<55:41, 42.84s/it]

# Парсинг

In [5]:
# сброс счетчика

counts_ = '0'

with connection_2.cursor() as cursor:
    cursor.execute(
    f'''UPDATE cars_count
    SET count = {counts_}'''
    )

In [6]:
query = "SELECT * FROM cars_count"
qer = pd.io.sql.read_sql(query, con = engine) 
i = int(qer['count'].values[0])
i

0

## Сам парсер

In [9]:
query = "SELECT * FROM cars_links"
qery = pd.io.sql.read_sql(query, con = engine) 
links = list(qery['links'].values)

query = "SELECT * FROM cars_count"
qer = pd.io.sql.read_sql(query, con = engine) 
i = int(qer['count'].values[0])
# Пустой ДФ

df = {}
df['Марка'] = []
df['Модель'] = []
df['Тип кузова'] = []
df['Год выпуска'] = []
df['Пробег'] = []
df['Объем двигателя'] = []
df['Тип двигателя'] = []
df['КПП'] = []
df['Привод'] = []
df['Комплектация'] = []
df['Цвет'] = []
df['Владельцев по ПТС'] = []
df['Стоимость'] = []
df['Дата объявления'] = []
df['номер объявления'] = []
df['Ссылка'] = []
df['Описание'] = []

options.headless = True
errrors = 0
for j,url in enumerate (links[i:]):
    driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
    driver.get(url)
    
        
    
    if j % 5 == 0 and j > 1:
              
        counts_ = str(i+j)

        with connection_2.cursor() as cursor:
            cursor.execute(
            f'''UPDATE cars_count
            SET count = {counts_}'''
            )
            
        
        df = pd.DataFrame(df)
        for q in range (len(df)):
            values_ = []
            for value in df.loc[q].values:
                values_.append(str(value))
            with connection_2.cursor() as cursor:
                    cursor.execute(
                    SQL_insert, [*values_]
                    )
        print(f'Данные и счетчик записаны на {j+i} позиции')
        df = {}
        df['Марка'] = []
        df['Модель'] = []
        df['Тип кузова'] = []
        df['Год выпуска'] = []
        df['Пробег'] = []
        df['Объем двигателя'] = []
        df['Тип двигателя'] = []
        df['КПП'] = []
        df['Привод'] = []
        df['Комплектация'] = []
        df['Цвет'] = []
        df['Владельцев по ПТС'] = []
        df['Стоимость'] = []
        df['Дата объявления'] = []
        df['номер объявления'] = []
        df['Ссылка'] = []
        df['Описание'] = []
        
        yandex()
        
    if is_bad_adv(i,j): 
        driver.close()
        time.sleep(2)
        continue 
    try:            
        params = get_params()
        df['Марка'].append(get_mark())
        df['Модель'].append(get_model())
        df['Тип кузова'].append(get_kuzov_type(params))
        df['Год выпуска'].append(get_year(params))
        df['Пробег'].append(get_probeg(params))
        df['Объем двигателя'].append(get_dvigatel(params))
        df['Тип двигателя'].append(get_dvig_type(params))
        df['КПП'].append(get_kpp(params))
        df['Привод'].append(get_privod(params))
        df['Комплектация'].append(get_complectation(params))
        df['Цвет'].append(get_color(params))
        df['Владельцев по ПТС'].append(get_vladelcev(params))
        df['Стоимость'].append(get_price())
        df['Дата объявления'].append(get_date())
        df['номер объявления'].append(url.split('_')[-1])
        df['Ссылка'].append(url)
        df['Описание'].append(get_view())
        
        print(f'{"***"*14} № {(j+i+1)} из {len(links)} {"***"*14}')
        if df["Марка"][-1] == None:
            print(f'Марка: {df["Марка"][-1]}')
        if df["Модель"][-1] == None:
            print(f'Модель: {df["Модель"][-1]}')
        if df["Тип кузова"][-1] == None:
            print(f'Тип кузова: {df["Тип кузова"][-1]}')
        if df["Год выпуска"][-1] == None:
            print(f'Год выпуска: {df["Год выпуска"][-1]}')
        if df["Пробег"][-1] == None:
            print(f'Пробег: {df["Пробег"][-1]}')
        if df["Объем двигателя"][-1] == None:
            print(f'Объем двигателя: {df["Объем двигателя"][-1]}')
        if df["Тип двигателя"][-1] == None:
            print(f'Тип двигателя: {df["Тип двигателя"][-1]}')
        if df["КПП"][-1] == None:
            print(f'КПП: {df["КПП"][-1]}')
        if df["Привод"][-1] == None:
            print(f'Привод: {df["Привод"][-1]}')
        if df["Комплектация"][-1] == None:
            print(f'Комплектация: {df["Комплектация"][-1]}')
        if df["Владельцев по ПТС"][-1] == None:
            print(f'Владельцев по ПТС: {df["Владельцев по ПТС"][-1]}')
        if df["Стоимость"][-1] == None:
            print(f'Стоимость: {df["Стоимость"][-1]}')
            errrors += 1
            if errrors >= 3:
                print('3 пустых подряд, что-то не так, останавливаем')
                break
            else:
                errrors = 0
        if df["Дата объявления"][-1] == None:
            print(f'Дата объявления: {df["Дата объявления"][-1]}')
        if df["номер объявления"][-1] == None:
            print(f'номер объявления: {df["номер объявления"][-1]}')
        if df["Описание"][-1] == None:
            print(f'Описание: {df["Описание"][-1]}')
        print(f'Ссылка: {df["Ссылка"][-1]}')
        
        S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
        driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
        driver.find_element('tag name' ,'body').screenshot(f'screenshots/{url.split("_")[-1]}.png')
        print('Скриншот сделан')
        driver.quit()
    
    except:
        print('Тут что-то пошло не так (ошибка с вылетом скрипта):')
        print(url)        
        break
        
    time.sleep(random.randint(10,19))

else:
    df = pd.DataFrame(df)
    for q in range (len(df)):
        values_ = []
        for value in df.loc[q].values:
            values_.append(str(value))
        with connection_2.cursor() as cursor:
                cursor.execute(
                SQL_insert, [*values_]
                )

    yandex()

    print('###'*22)
    print('#'+' '*64+'#')
    print('#'+ ' ' *20 + 'СКРИПТ ОКОНЧИЛ РАБОТУ!' + ' '*22 + '#')
    print('#'+' '*64+'#')
    print('###'*22)

****************************************** № 296 из 583 ******************************************
Ссылка: https://www.avito.ru/izhevsk/avtomobili/mercedes-benz_gle-klass_2015_2564286935
Скриншот сделан
****************************************** № 297 из 583 ******************************************
Ссылка: https://www.avito.ru/izhevsk/avtomobili/renault_logan_2009_2560336036
Скриншот сделан
****************************************** № 298 из 583 ******************************************
Ссылка: https://www.avito.ru/izhevsk/avtomobili/vaz_lada_granta_2017_2511069114
Скриншот сделан
****************************************** № 299 из 583 ******************************************
Комплектация: None
Ссылка: https://www.avito.ru/izhevsk/avtomobili/volkswagen_touareg_2012_2395359597
Скриншот сделан
****************************************** № 300 из 583 ******************************************
Ссылка: https://www.avito.ru/izhevsk/avtomobili/vaz_lada_granta_2014_2363591175
Скриншот сдел

# Предобработка

In [25]:
query = "SELECT * FROM cars_ishevsk"
df = pd.io.sql.read_sql(query, con = engine)
df = df[df.columns[1:]]
df.head()

,mark,model,kuzov,year,km,engine_volume,engine_type,kpp,privod,complectation,color,owners,price,date,ad_number,url_link,description
0,ВАЗ (LADA),Vesta,Седан,2019,29283,1.6,Бензин,Механика,Передний,Luxe,Белый,1,749000.0,9.9.2022,2501257010,https://www.avito.ru/izhevsk/avtomobili/lada_v...,Автомобиль в отличном состоянии ! Автомобиль н...
1,ГАЗ,ГАЗель Next,Фургон,2020,255289,2.8,Дизель,Механика,Задний,Базовая,Белый,1,2085000.0,9.9.2022,2554843038,https://www.avito.ru/izhevsk/avtomobili/gaz_ga...,"🟢 Все растентовки; 🟢 1 владелец; 🟢 Надежный, д..."
2,Audi,Q3,Внедорожник,2012,150000,2.0,Бензин,Робот,Полный,Base,Чёрный,3,1279000.0,9.9.2022,2556895837,https://www.avito.ru/izhevsk/avtomobili/audi_q...,На все наши автомобили распространяется технич...
3,ВАЗ (LADA),Granta,Лифтбек,2015,104000,1.6,Бензин,Механика,Передний,None,Серый,2,419000.0,9.9.2022,2527641130,https://www.avito.ru/izhevsk/avtomobili/vaz_la...,"В комплектации: усилитель руля, кондиционер, э..."
4,Renault,Duster,Внедорожник,2018,44124,2.0,Бензин,Механика,Полный,None,Серый,1,1205000.0,9.9.2022,2363371296,https://www.avito.ru/izhevsk/avtomobili/renaul...,✔На автомобиль возможна АВТОРАССРОЧКА ✔1 владе...


In [4]:
df.duplicated(subset=['ad_number']).sum()

1358

In [26]:
df = df.drop_duplicates(subset=['ad_number'])
cols = df.columns
for col in cols:
    print(f'{col}: {df[col].isna().sum()}')

mark: 0
model: 0
kuzov: 0
year: 0
km: 0
engine_volume: 0
engine_type: 0
kpp: 0
privod: 0
complectation: 0
color: 0
owners: 0
price: 0
date: 0
ad_number: 0
url_link: 0
description: 0


In [13]:
for col in cols:
    print(f'{col}: {(df[col] == "nan").sum()}')

mark: 0
model: 0
kuzov: 0
year: 0
km: 0
engine_volume: 0
engine_type: 0
kpp: 0
privod: 0
complectation: 0
color: 0
owners: 0
price: 0
date: 0
ad_number: 0
url_link: 0
description: 0


In [18]:
for col in cols:
    print(f'{col}: {(df[col] == "None").sum()}')

mark: 0
model: 0
kuzov: 0
year: 0
km: 0
engine_volume: 0
engine_type: 0
kpp: 0
privod: 0
complectation: 496
color: 0
owners: 0
price: 0
date: 0
ad_number: 0
url_link: 0
description: 0


In [33]:
df = df[df['price'] != 'nan']
df = df[df['date'] != 'None']
df = df[df['engine_type'] != 'None']
df = df[df['price'] < np.percentile(df['price'], 98)]
df['price'] = df['price'].apply(float)
df['datetime'] = df['date'].apply(get_date_type)

In [35]:
df.shape

(1852, 18)

In [36]:
df.to_excel('cars_ishevsk.xlsx', index=0)